# UR5 Kinematics simulation

We will use th "ur5_e_moveit_config" package in "src/fmauch_universal_robot" folder for simulation purposes. 

Detailed information will be found in readme file on: https://github.com/fmauch/universal_robot/tree/melodic-devel-staging

We will be able to simulate the forward and Inverse Kinematics

## Forward Kinematics
Be sure you have installed "rqt-joint-trajectory-controller" package:

In [ ]:
sudo apt-get install ros-melodic-rqt-joint-trajectory-controller

### Usage with Gazebo Simulation

To bring up the simulated robot in Gazebo, run:

In [ ]:
roslaunch ur_e_gazebo ur5e.launch

In [ ]:
rosrun rqt_joint_trajectory_controller rqt_joint_trajectory_controller

<img src="./Images/2_gazebo_joint_controller.png">

## Inverse Kinematics
For inverse kinematics you can use moveIt!
### MoveIt! with a simulated robot

Again, you can use MoveIt! to control the simulated robot.

Tutorial
http://docs.ros.org/en/melodic/api/moveit_tutorials/html/index.html

Videos
https://www.youtube.com/watch?v=b4T577d39dE

https://www.youtube.com/watch?v=BxCik8OI1Fw

As MoveIt! seems to have difficulties with finding plans for the UR with full joint limits [-2pi, 2pi], there is a joint_limited version using joint limits restricted to [-pi,pi]. In order to use this joint limited version, simply use the launch file arguments 'limited', i.e.:

In [ ]:
roslaunch ur_e_gazebo ur5e.launch limited:=true

For setting up the MoveIt! nodes to allow motion planning run:

In [ ]:
roslaunch ur5_e_moveit_config ur5_e_moveit_planning_execution.launch sim:=true limited:=true

For starting up RViz with a configuration including the MoveIt! Motion Planning plugin run:

In [ ]:
roslaunch ur5_e_moveit_config moveit_rviz.launch config:=true

<img src="./Images/2_gazebo_rviz_IK.png">

You can:
- Plan: with Motion Planning you can view the results in rviz
- execute: the planned movement is executed in gazebo

### Specific package for Kinematics control

We can create a specific package for kinematics control.

Create the ur5control package with dependencies:
- moveit_core 
- moveit_visual_tools 
- moveit_ros_planning_interface 
- interactive_markers 
- tf2_geometry_msgs

In [ ]:
catkin_create_pkg ur5control rospy std_msgs moveit_core moveit_visual_tools moveit_ros_planning_interface interactive_markers tf2_geometry_msgs

create a specific python file to control the UR5 arm movement in:
- Forward Kinematics --> Joints control
- Inverse Kinematics --> Pose Control

Run first the gazobo model, planning execution and moveit rviz

In [ ]:
roslaunch ur_e_gazebo ur5e.launch limited:=true
roslaunch ur5_e_moveit_config ur5_e_moveit_planning_execution.launch sim:=true limited:=true
roslaunch ur5_e_moveit_config moveit_rviz.launch config:=true

### Joints control
We create the "ur5e_joints_control1.py" to control the different link angles:



In [ ]:
rosrun ur5control ur5e_joints_control1.py 

<img src="./Images/2_control_joints1.png">

### Pose control

We create the "ur5e_pose_control1.py" to control the pose target:

In [ ]:
rosrun ur5control ur5e_pose_control1.py 

<img src="./Images/2_control_pose1.png">

### External Kinematics control

If you want to control the ur5e kinematics using an external device, you will need to communicate with it according to:

<img src="./Images/2_pen_control_node.png">

We will create 2 python files:
- "ur5e_pose2robot.py" to define the /ur5e_node
- "ur5e_pen_control.py" to define the /pen_node

We can create a "ur5e_bringup_sim.launch" launch file to start the robot arm in gazebo and the planning execution

In [ ]:
<launch>
  <arg name="sim" default="true" />
  <arg name="limited" default="true"/>
  <!-- Launch ur5e -->
   <include file="$(find ur_e_gazebo)/launch/ur5e.launch">
    <arg name="limited" default="$(arg limited)"/>
  </include>
  <!-- Launch planning execution -->
  <include file="$(find ur5_e_moveit_config)/launch/ur5_e_moveit_planning_execution.launch">
    <arg name="limited" default="$(arg limited)"/>
    <arg name="sim" default="$(arg sim)"/>
  </include>
</launch>

we have created a "ur5e_pose_control.launch" to start the 2 nodes to control the pose with a pen device

In [ ]:
<launch>

  <node name="pen_control" pkg="ur5control" type="ur5e_pen_control.py" output="screen">
  </node>
  <node name="ur5e_node" pkg="ur5control" type="ur5e_pose2robot.py" output="screen">
  </node>

</launch>

We can start all the nodes:

In [ ]:
roslaunch ur5control ur5e_bringup.launch
roslaunch ur5control ur5e_pose_control.launch

<img src="./Images/1_bringup1.png">